In [1]:
import numpy as np 
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import pathlib
import tensorflow as tf
from tensorflow.keras import layers

In [5]:
from pathlib import Path
import imghdr
import os
data_dir = "./Datasets/ImageRetrieval/Train/"
# add there all your images file extensions
image_extensions = [".png", ".jpg",".jpeg"]

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            os.remove('./'+str(filepath))
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            os.remove('./'+str(filepath))
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")


Datasets\ImageRetrieval\Train\Donut\Donut (16).jpeg is not an image
Datasets\ImageRetrieval\Train\Hot Dog\Hot Dog - Train (11).jpeg is not an image
Datasets\ImageRetrieval\Train\Hot Dog\Hot Dog - Train (39).jpeg is not an image
Datasets\ImageRetrieval\Train\Hot Dog\Hot Dog - Train (42).jpeg is not an image
Datasets\ImageRetrieval\Train\Hot Dog\Hot Dog - Train (59).jpeg is a webp, not accepted by TensorFlow
Datasets\ImageRetrieval\Train\Hot Dog\Hot Dog - Train (73).jpeg is not an image
Datasets\ImageRetrieval\Train\Hot Dog\Hot Dog - Train (75).jpeg is not an image


In [2]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory('./Datasets/ImageRetrieval/Train', batch_size=128,
                                                               label_mode='categorical', image_size=(224, 224), seed=1)
test_ds = tf.keras.preprocessing.image_dataset_from_directory('./Datasets/ImageRetrieval/Test', batch_size=128,
                                                               label_mode='categorical', image_size=(224, 224), seed=1)
val_ds = tf.keras.preprocessing.image_dataset_from_directory('./Datasets/ImageRetrieval/Valid', batch_size=128,
                                                             label_mode='categorical', image_size=(224, 224), seed=1)


Found 14811 files belonging to 10 classes.
Found 1500 files belonging to 10 classes.
Found 3500 files belonging to 10 classes.


In [3]:
def format_image(image, label):
    image = tf.image.resize(image, (224, 224))/255.0

    image = preprocess_input(image)
    return image, label

In [4]:
train_ds = train_ds.map(format_image).prefetch(256)
test_ds = test_ds.map(format_image).prefetch(256)
val_ds = val_ds.map(format_image).prefetch(256)


## Third pooling layer features

In [5]:
base_model = VGG16(weights='imagenet')
model = Model(inputs=base_model.input,
              outputs=base_model.get_layer('block3_pool').output)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
from tqdm import tqdm
train_features = []
for items in tqdm(train_ds):
  predictions = model.predict(items[0], verbose=False)
  for prediction in predictions:
    train_features.append(prediction.flatten())


100%|██████████| 116/116 [02:38<00:00,  1.37s/it]


In [13]:
len(train_features)

14811

In [ ]:
val_features = []
for items in tqdm(val_ds):
  predictions = model.predict(items[0])
  for prediction in predictions:
    val_features.append(prediction.flatten())


In [ ]:
test_features = []
for items in tqdm(test_ds):
  predictions = model.predict(items[0])
  for prediction in predictions:
    test_features.append(prediction)


In [ ]:
full_dataset = np.concatenate((train_features, val_features))


In [9]:
import pickle

# with open('./Datasets/ImageRetrieval/ProcessedFeatures/train_feat_pool3.pkl', 'wb') as f:
#   pickle.dump(train_features, f)
# with open('./Datasets/ImageRetrieval/ProcessedFeatures/test_feat_pool3.pkl', 'wb') as f:
#   pickle.dump(test_features, f)
with open('./Datasets/ImageRetrieval/ProcessedFeatures/val_feat_pool3.pkl', 'wb') as f:
  pickle.dump(val_features, f)


In [2]:
import pickle

with open('./Datasets//ImageRetrieval/ProcessedFeatures/train_feat_pool3.pkl', 'rb') as f:
  train_features = pickle.load(f)
# with open('./ImageRetrieval/ProcessedFeatures/test_feat_pool3.pkl', 'rb') as f:
#   test_features = pickle.pickle.load(f)
# with open('./Datasets//ImageRetrieval/ProcessedFeatures/val_feat_pool3.pkl', 'rb') as f:
#   val_features = pickle.load(f)


In [6]:
full_dataset = np.concatenate((train_features, val_features))

In [7]:
from sklearn.neighbors import KDTree

tree = KDTree(train_features)


In [ ]:
import pickle

with open('./kb_tree.pkl','wb') as f:
  pickle.dump(tree, f)

## Last pooling layer features